In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_prep import create_x_y_data

In [2]:
#import data
X_train, y_train, X_test, y_test, X, y = create_x_y_data()

print(X_train.shape)
print(X_test.shape)

(93, 1984)
(44, 1984)


In [3]:
# import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier
from xgboost import XGBClassifier

# Using the ConsensML class

## Overview:

1) Finds intersection of features deemed important by Lasso Regression, XGBoost, and Random Forest

2) For each model: tunes paramters on 5 fold CV -> fits tuned model on 5 fold CV -> identifies features deemed "important" in all 5 folds

3) Runs a random forest Boruta test

4) Combines the discovered features in five different ways (Boruta only, Boruta + Intersection, Boruta + Lasso KF, Boruta + Lasso/XGB KF, Intersection + Lasso/XGB KF) then fits a model to see which features are best

In [4]:
from consensus import ConsensusML

In [6]:
lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=10000)
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=8, n_jobs=-1)
xgb = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 700, random_state=8, n_jobs=-1)

cm = ConsensusML(lasso, rf, xgb, X, y) #call model (should just initialize then go to cm.add_models)

model_dict = cm.feat_selection3(X_train, X_test, y_train, y_test) #fit models -> log loss/genes/total genes

gene_intersection, lasso_weights, xgb_feature_importance = cm.feature_intersection_weights() #find intersection


param_list =  [ {'C': range(1, 4, 1)}, {'max_depth': range(3, 7), 'n_estimators': range(200, 800, 200)},
               {'max_depth': range(3, 10)} ] #list of paramters for CV (lasso, xgb, rf)

cm.kfold_tune(param_list) #use the param list to tune the models

performances, weights = cm.kfold_weights() # fit tuned models on kfold --> return weight/log loss

cm.boruta_selection() # run boruta test

final_feats = cm.feature_consensus() # intersection of all previous tests

model = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=8, n_jobs=-1) #call model to compare

feature_performances, best_dataframe = cm.best_combo(model) #find the best combination of consensus

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.7min finished
/Users/david/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   26.0s finished
/Users/david/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Fitting 5 folds for each of 7 candidates, totalling 35 fits


...

/Users/david/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	98 / 100
Confirmed: 	85
Tentative: 	59
Rejected: 	1840


/Users/david/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	99 / 100
Confirmed: 	85
Tentative: 	59
Rejected: 	1840


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	85
Tentative: 	14
Rejected: 	1840
(137, 85)
[('Boruta', 0.2320984242668982)]


NameError: name 'np' is not defined

In [8]:
feature_performances, best_dataframe = cm.best_combo(model)

[('Boruta', 0.17996442482125813)]


In [12]:
feature_performances

{'Boruta': {'Log Loss': 0.17996442482125813,
  'Model Weights': array([0.00316054, 0.02037491, 0.00329472, 0.00359504, 0.00146195,
         0.01374504, 0.0074932 , 0.01502901, 0.00640196, 0.00302314,
         0.03476624, 0.00764533, 0.06997857, 0.00773679, 0.00834866,
         0.00320989, 0.03149242, 0.05610372, 0.00709896, 0.02344204,
         0.00195659, 0.03590717, 0.01051769, 0.00507905, 0.0110827 ,
         0.0046177 , 0.00638565, 0.00554783, 0.03865334, 0.00219166,
         0.02310788, 0.00239922, 0.00483039, 0.00576532, 0.01214881,
         0.00641859, 0.03958241, 0.03898611, 0.01622602, 0.00417117,
         0.01505853, 0.00725237, 0.01098657, 0.00360894, 0.00935465,
         0.01016532, 0.05718348, 0.00562912, 0.00926734, 0.05008643,
         0.00418294, 0.01508668, 0.006296  , 0.01965817, 0.01536025,
         0.04671163, 0.00563171, 0.00992244, 0.01120009, 0.06483991,
         0.01412703, 0.00281009, 0.00260289])}}